In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

BATCH_SIZE = 20
NUM_EPOCHS = 300
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'baseline-mlp-%dsamples' % (SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D)

In [3]:
from local_enc import *
from global_oneshot import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_z)

In [ ]:
from os_ep import *
train_baseline(models, EP, optimizer, Data, Model_Params, Train_Params)

epoch: 0\300 (8s),  eubo: -586.026,  elbo: -692.205,  ess: 1.093
epoch: 1\300 (8s),  eubo: -496.624,  elbo: -570.970,  ess: 1.116
epoch: 2\300 (8s),  eubo: -433.592,  elbo: -480.539,  ess: 1.204
epoch: 3\300 (7s),  eubo: -403.787,  elbo: -439.192,  ess: 1.251
epoch: 4\300 (7s),  eubo: -387.014,  elbo: -414.471,  ess: 1.325
epoch: 5\300 (8s),  eubo: -378.872,  elbo: -404.414,  ess: 1.358
epoch: 6\300 (8s),  eubo: -360.463,  elbo: -379.935,  ess: 1.442
epoch: 7\300 (8s),  eubo: -354.090,  elbo: -371.078,  ess: 1.531
epoch: 8\300 (8s),  eubo: -344.948,  elbo: -358.381,  ess: 1.540
epoch: 9\300 (8s),  eubo: -336.561,  elbo: -347.627,  ess: 1.582
epoch: 10\300 (8s),  eubo: -326.535,  elbo: -337.400,  ess: 1.536
epoch: 11\300 (8s),  eubo: -318.676,  elbo: -330.485,  ess: 1.529
epoch: 12\300 (8s),  eubo: -311.779,  elbo: -324.312,  ess: 1.543
epoch: 13\300 (8s),  eubo: -300.245,  elbo: -313.616,  ess: 1.553
epoch: 14\300 (8s),  eubo: -295.677,  elbo: -310.063,  ess: 1.531
epoch: 15\300 (8s), 

In [ ]:
torch.save(enc_z.state_dict(), '../weights/enc-z-%s' + PATH)
torch.save(oneshot_eta.state_dict(), '../weights/oneshot-eta-%s' + PATH)

In [ ]:
def test(models, objective, Data, Model_Params, Train_Params):
    (NUM_EPOCHS, NUM_DATASETS, S, B, CUDA, device, path) = Train_Params
    SubTrain_Params = (device, S, B) + Model_Params
    ##(N, K, D, mcmc_size) = Model_Params
    indices = torch.randperm(NUM_DATASETS)
    batch_indices = indices[0*B : (0+1)*B]
    obs = Data[batch_indices]
    obs = shuffler(obs).repeat(S, 1, 1, 1)
    if CUDA:
        obs =obs.cuda().to(device)
    loss, metric_step, reused = objective(models, obs, SubTrain_Params)
    return obs, metric_step, reused

In [ ]:
BATCH_SIZE_TEST = 50
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
obs, metric_step, reused = test(models, EP, Data, Model_Params, Train_Params_Test)
(q_eta, _, q_z, _, _, _) = reused

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap